## <font color='#0000FF'>MSC_DA_CA2 - INTEGRATEDCA2<font color='#1ABC9D'>
### <font color='#'>**Advanced Data Analytics  & Big Data Storage and Processing**
### <font color='#1ABC9C'>**Lecturer(s): David McQuaid and Muhammad Iqbal**
------
<font color='#1ABC9C'>**Student name / ID** // Rosilene Francisca da Silva - 2021090

### Configure Apache Spark
Start Spark Session:
Set up PySpark, including necessary packages for MongoDB and MySQL connections.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Database Comparative Analysis") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/twitter_data.tweets") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/twitter_data.tweets") \
    .config("spark.jars", "/home/hduser/Downloads/mysql-connector-j-8.0.33.jar") \
    .getOrCreate()

24/05/05 16:00:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Twitter Data Analysis") \
    .getOrCreate()

# Load the dataset from HDFS
tweets_df = spark.read.csv("hdfs:///user/hduser/twitter_project/ProjectTweets.csv", header=True, inferSchema=True)
tweets_df.show(2, truncate=False)

24/05/05 16:00:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|0  |1467810369|Mon Apr 06 22:19:45 PDT 2009|NO_QUERY|_TheSpecialOne_|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|
+---+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|1  |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY|scotthamilton  |is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |
|2  |1467810917|Mon Apr 06 22:19:53 PDT 2009|NO_QUERY|mattycus       |@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                          |
+---+----------+----------------------------+--------+-----------

In [3]:
# Define column names
column_names = ["id", "user_id", "date", "query_status", "user_handle", "tweet_text"]

# Rename the columns
tweets_df = tweets_df.toDF(*column_names)
tweets_df.show(5, truncate=False) # Using truncate=False allows to see the full content.

+---+----------+----------------------------+------------+-------------+---------------------------------------------------------------------------------------------------------------+
|id |user_id   |date                        |query_status|user_handle  |tweet_text                                                                                                     |
+---+----------+----------------------------+------------+-------------+---------------------------------------------------------------------------------------------------------------+
|1  |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY    |scotthamilton|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!|
|2  |1467810917|Mon Apr 06 22:19:53 PDT 2009|NO_QUERY    |mattycus     |@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                      |
|3  |1467811184|Mon Apr 06 22:19:57 PDT 2009|NO_QUERY    |ElleCTF      |my 

#### Adding a New Column with the Current Timestamp
The `inserted_at` column will contain the timestamp indicating when each row was processed.

In [4]:
# add or modify data fields
from pyspark.sql.functions import current_timestamp

# Adding a current timestamp column for the insert time
tweets_df = tweets_df.withColumn("inserted_at", current_timestamp())
tweets_df.show(2, truncate=False)

+---+----------+----------------------------+------------+-------------+---------------------------------------------------------------------------------------------------------------+--------------------------+
|id |user_id   |date                        |query_status|user_handle  |tweet_text                                                                                                     |inserted_at               |
+---+----------+----------------------------+------------+-------------+---------------------------------------------------------------------------------------------------------------+--------------------------+
|1  |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY    |scotthamilton|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!|2024-05-05 16:01:01.533403|
|2  |1467810917|Mon Apr 06 22:19:53 PDT 2009|NO_QUERY    |mattycus     |@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out 

## Database Comparative Analysis

### Export Data from Spark to Databases

In [5]:
tweets_df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/twitter_data',
    driver='com.mysql.cj.jdbc.Driver',
    dbtable='tweets',
    user='root',
    password='password'
).mode('append').save()

Py4JJavaError: An error occurred while calling o60.save.
: java.lang.ClassNotFoundException: com.mysql.cj.jdbc.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at 

In [ ]:
!pip install mysql-connector-python
import mysql.connector

In [ ]:
import mysql.connector

# Establish a MySQL connection
config = {
    "user": "root",  # MySQL username
    "password": "password",  # MySQL password
    "host": "localhost",  # Host where MySQL is running
    "database": "twitter_data",  # Database name
    "raise_on_warnings": True
}

# Create a connection object
conn = mysql.connector.connect(**config)

# Create a cursor object using the connection
cursor = conn.cursor()

# Query to select data
query = "SELECT * FROM tweets LIMIT 5;"

# Execute the query
cursor.execute(query)

# Fetch all rows from the last executed statement using fetchall()
rows = cursor.fetchall()

# Print rows
for row in rows:
    print(row)

# Close the cursor and the connection
cursor.close()
conn.close()

#### Export to MySQL:
Use JDBC to write the DataFrame to MySQL

####  Include the JDBC Driver in Your Spark Session and Loading the Driver During Runtime:
As PySpark session is already running and I cannot restart it with the --jars or --packages options, I decided dynamically add the JDBC driver to the classpath using the SparkContext:

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder \
    .appName("Twitter Data Analysis") \
    .getOrCreate()

sc = spark.sparkContext
sc.addPyFile("/path/to/mysql-connector-java-8.0.xx.jar")

In [ ]:
# DataFrame loaded with the data to write to MySQL
tweets_df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/twitter_data',  # Replace 'twitter_data' with actual database name
    driver='com.mysql.cj.jdbc.Driver',
    dbtable='tweets',  # Replace 'tweets' with actual table name
    user='root',  # Replace with actual MySQL username
    password='password'  # Replace with your actual MySQL password
).mode('append').save()

#### Export to MongoDB:
Write the DataFrame to MongoDB:

In [ ]:
tweets_df.write.format("mongo").option("uri", "mongodb://localhost/twitter_data.tweets").mode("append").save()

### Configure YCSB for MySQL and Cassandra:
Edit the YCSB properties files for MySQL and Cassandra to include the connection details, such as host, port, database/keyspace, and authentication info.

### Run YCSB Benchmarks
#### Load Data and Perform Tests:
#### For MySQL

In [ ]:
./bin/ycsb load jdbc -P workloads/workloada -p db.driver=com.mysql.cj.jdbc.Driver -p "db.url=jdbc:mysql://localhost/twitter_data?user=your_mysql_user&password=your_mysql_password"
./bin/ycsb run jdbc -P workloads/workloada -p db.driver=com.mysql.cj.jdbc.Driver -p "db.url=jdbc:mysql://localhost/twitter_data?user=your_mysql_user&password=your_mysql_password"

#### For MongoDB:

In [ ]:
./bin/ycsb load mongodb -P workloads/workloada -p "mongodb.url=mongodb://127.0.0.1:27017/twitter_data.tweets"
./bin/ycsb run mongodb -P workloads/workloada -p "mongodb.url=mongodb://127.0.0.1:27017/twitter_data.tweets"

### Analyze Results
Compare Performance Metrics:
Analyze the output from YCSB, focusing on metrics such as throughput, latency, and error rates. Use this data to compare the performance characteristics of MySQL and Cassandra under similar loads.

### Document Findings
Report Generation:
Document the setup, methodology, results, and analysis in a comprehensive report. Use charts and graphs to illustrate performance comparisons where applicable.

### Step 2: Data Processing in PySpark
Before exporting the data to any database, you can apply necessary data transformations, filtering, or aggregations in PySpark.

In [ ]:
# Example of filtering tweets containing a specific keyword
filtered_tweets_df = tweets_df.filter(tweets_df.tweet.contains("important"))

# might also want to add or modify data fields
from pyspark.sql.functions import current_timestamp

# Adding a current timestamp column for the insert time
final_df = filtered_tweets_df.withColumn("inserted_at", current_timestamp())

In [ ]:
#pip uninstall matplotlib -y
!pip install matplotlib

In [ ]:
#Importing the libraries.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import cm
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Command to display all columns in the file.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Importing the Dataset 
tweets_df = pd.read_csv('ProjectTweets.csv')
tweets_df.head()

In [ ]:
# Set column's names
tweets_df.columns = ['index', 'user_id', 'date', 'query_status', 'user_handle', 'tweet_text']
tweets_df.head(2)

After load the CSV file "ProjectTweets.csv" was noted that it has no headers. Renaming columns in a DataFrame makes the code cleaner and easier to comprehend and maintain. 

In [ ]:
tweets_df.shape

In [ ]:
tweets_df.info()

Based on the first basic details, the dataset appears to be a collection of tweets, commonly can be use in sentiment analysis or other natural language processing tasks. 

Dataset Size and Integrity:
The dataset contains approximately 1.6 million entries (1599999 entries, indexed from 0 to 1599998).
Each column has the same number of non-null entries (1599999), indicating there are no missing values in any of the columns.
Memory Usage: The dataframe is consuming approximately 73.2 MB of memory, which is relevant for understanding the computation resources needed to process this dataset.

In [ ]:
tweets_df.dtypes

In [ ]:
print('length of data is', len(tweets_df))

In [ ]:
tweets_df.isnull().values.any()

#### Filtering Data
Filter rows based on some criteria. For example, to see tweets that have no query status:

In [ ]:
no_query_tweets = tweets_df[tweets_df['query_status'] == 'NO_QUERY']
print(no_query_tweets.head())

In [ ]:
tweets_df.query_status.value_counts()

The feature `query_status` shows whether any specific query key phrase has been used to collect the tweets; nonetheless, 100% of the entries in this column have the value "NO_QUERY."

#### Display the Unique Values

In [ ]:
for column in tweets_df.columns:
    num_unique_values = tweets_df[column].nunique()
    print(f"Number of unique values in '{column}': {num_unique_values}")

In [ ]:
def return_unique_values(data_frame):
    unique_dataframe = pd.DataFrame()
    unique_dataframe['Features'] = data_frame.columns
    uniques = []
    for col in data_frame.columns:
        u = data_frame[col].nunique()
        uniques.append(u)
    unique_dataframe['Uniques'] = uniques
    return unique_dataframe

unidf = return_unique_values(tweets_df)
print(unidf)

##### To begin with, dropping the columns that are unnecessary for the particular goal of sentiment analysis.The `index` and `query_status`, columns in this instance.

In [ ]:
# Dropping the 'index' and 'query_status' columns
tweets_df.drop(columns=['index', 'query_status'], inplace=True)

# Display the DataFrame to confirm changes
tweets_df.head(2)

#### DateTime Parsing - Convert Date Format
The `date` column contains date and time information as a string, let's to convert it into a DateTime for easier manipulation and to facilitate time series operations.

In [ ]:
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce', format='%a %b %d %H:%M:%S PDT %Y')
tweets_df.head(5)

In [ ]:
print(tweets_df['date'].dtype)

In [ ]:
tweets_df.isnull().values.any()

In [ ]:
tweets_df.describe()

The summary statistics for the user_id and date columns provide many insights into the user's activity and timing characteristics during the data collecting period. The user_id has a mean of around 1.998 billion, indicating a wide range of values, potentially due to a big user base or a wide encoding range for user identification. The difference between the minimum (1.467811e+09) and maximum (2.329206e+09) values, with a standard deviation of around 193.6 million, indicates significant variability in user ID assignments, which could be due to different periods of user registration or system changes affecting ID allocation. The date column ranges from April 6, 2009 to June 25, 2009, with the median occurring on June 2, 2009. 

The dataset may have captured seasonal user behaviour or events because of its temporal distribution, which shows that it covers almost three months in late spring to early summer. The mid-June and end-of-May 25th and 75th percentiles, respectively, point to a concentration of recordings around these dates, which may correspond with particular events or promotions that encourage user participation during these times.

#### Text Analysis
Analyzing the tweet texts, let's counting the number of words in each tweet

In [ ]:
# Calculate the length of each tweet and the word count
tweets_df['text_length'] = tweets_df['tweet_text'].apply(len)
tweets_df['word_count'] = tweets_df['tweet_text'].apply(lambda x: len(x.split()))

# Display these new metrics
print(tweets_df[['tweet_text', 'text_length', 'word_count']])

In [ ]:
tweets_df['word_count'] = tweets_df['tweet_text'].apply(lambda x: len(str(x).split()))
tweets_df.head()

#### Sentiment Analysis

Using a simple library like TextBlob, to get a rough idea of the sentiment of each tweet.

In [ ]:
#!pip install textblob

In [ ]:
from textblob import TextBlob

# Calculate sentiment polarity
tweets_df['sentiment'] = tweets_df['tweet_text'].apply(lambda text: TextBlob(text).sentiment.polarity)

# Display tweets with their sentiment
print(tweets_df[['tweet_text', 'sentiment']])

### Visualization
Visualizing aspects of the data can further aid in understanding the distribution and relationships.

#### Time Series Plot of Tweet Frequency

In [ ]:
import matplotlib.pyplot as plt

# Plotting the number of tweets over time
tweets_df.set_index('date').resample('S').size().plot()
plt.title('Tweet Frequency Over Time')
plt.xlabel('Time')
plt.ylabel('Number of Tweets')
plt.show()

#### Histogram of Text Length and Word Count

In [ ]:
tweets_df['text_length'].plot(kind='hist', title='Distribution of Tweet Text Length', bins=20, alpha=0.7)
plt.xlabel('Text Length')
plt.show()

tweets_df['word_count'].plot(kind='hist', title='Distribution of Tweet Word Count', bins=20, alpha=0.7)
plt.xlabel('Word Count')
plt.show()

#### Distributions
Explore the distribution of key variables using histograms or box plots to understand their central tendencies and spread.

#### Histograms: Useful for visualizing the distribution of numerical data.

In [ ]:
user_ids = tweets_df['user_id']

# Calculate histogram data
counts, bin_edges = np.histogram(user_ids, bins=10)

# Create a histogram
plt.figure(figsize=(10, 6))  # Make it large enough for clarity
plt.bar(bin_edges[:-1], counts, width=np.diff(bin_edges), edgecolor='black', align='edge', color='skyblue')
plt.title('Distribution of User IDs')
plt.xlabel('User ID Bins')
plt.ylabel('Frequency')
plt.grid(True)  

# Remove all unnecessary tick marks and frame pieces
plt.tick_params(top=False, right=False, left=True, bottom=True, labelleft=True, labelbottom=True)
for spine in plt.gca().spines.values():
    if spine.spine_type not in ['bottom', 'left']:  
        spine.set_visible(False)

plt.show()

#### Box Plots: Good for detecting outliers and understanding the distribution's quartiles.

In [ ]:
import matplotlib.pyplot as plt

# Create a box plot
plt.figure(figsize=(8, 6)) 
plt.boxplot(user_ids, vert=False, widths=0.7, patch_artist=True, flierprops={'marker':'o', 'color':'red', 'markersize':5})
plt.title('User ID Distribution')
plt.xlabel('User IDs')
plt.yticks([])  

# Enhance data-ink ratio
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)  # Remove left spine if y-axis ticks are not informative
plt.grid(True, linestyle='--', which='major', color='gray', alpha=0.5)

plt.show()

#### Plot histograms and density plots of user_id to understand user engagement and presence. 
Because the histogram helps to visualize how active users are in terms of the number of tweets they post. And the density plot provides a clear view of the distribution's shape, highlighting the typical user activity levels without the binning process of a histogram.

First, set up correctly for visualising user activity using user_id and analysing the frequency of user interactions.

In [ ]:
# Count the frequency of each user_id
user_activity = tweets_df['user_id'].value_counts()

import matplotlib.pyplot as plt

# Create a histogram of user activity
plt.figure(figsize=(10, 6))
plt.hist(user_activity.values, bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram of User Activity')
plt.xlabel('Number of Tweets per User')
plt.ylabel('Frequency')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)  # Minimal grid use
plt.show()

In [ ]:
import seaborn as sns

# Create a density plot
plt.figure(figsize=(10, 6))
sns.kdeplot(user_activity.values, shade=True, color='blue', alpha=0.7)
plt.title('Density Plot of User Activity')
plt.xlabel('Number of Tweets per User')
plt.ylabel('Density')
plt.show()

#### Pair Plots
Use pair plots to visualise the relationships and distributions of each pair of variables in the data.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a pair plot with a clean and minimalist design
sns.set(style="white")  # Sets the style of the plot to a simple white background
pairplot = sns.pairplot(tweets_df, diag_kind='kde', plot_kws={'alpha': 0.6, 's': 80, 
                                                              'edgecolor': 'k'}, corner=True)
for i in range(len(pairplot.axes)):
    for j in range(len(pairplot.axes)):
        if i != j:
            pairplot.axes[i][j].set_visible(False)
            if i == j:
                pairplot.axes[i][j].set_ylabel('Density')

# Add titles and labels
plt.subplots_adjust(top=0.9)
pairplot.fig.suptitle('Pairwise Plots of User Metrics', fontsize=16)

plt.show()

#### Correlation Matrix
Investigate the correlations between numerical variables to better understand the links between the fields in the dataset.
Creating a correlation matrix is an excellent way to visually and quantitatively investigate the correlations between numerical variables in your dataset.

In [ ]:
import pandas as pd

# Compute the correlation matrix
corr_matrix = tweets_df.corr()

# Print the correlation matrix
print(corr_matrix)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap from the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True, linewidths=.5)

# Enhancing the heatmap
plt.title('Correlation Matrix of Variables')
plt.yticks(rotation=0)  # Rotate y-labels for better readability
plt.xticks(rotation=90)  # Rotate x-labels for better readability
plt.show()

### Text Data Analysis

#### Sentiment Analysis: 
Planing to forecast sentiment trends, let's first need to compute the sentiment scores of the tweets.

In [ ]:
from textblob import TextBlob
tweets_df['sentiment'] = tweets_df['tweet_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

#### Aggregate Sentiment Over Time: 
Resample the sentiment data to daily, and calculate mean sentiment.

In [ ]:
daily_sentiment = tweets_df['sentiment'].resample('D').mean()
daily_sentiment.plot(title='Daily Sentiment Score')
plt.xlabel('Date')
plt.ylabel('Sentiment Score')
plt.show()

### Time Series Analysis and Forecasting

#### Set Date as Index: For time series analysis, it's useful to have the date as the DataFrame index

In [ ]:
tweets_df.set_index('date', inplace=True)
tweets_df.head(2)

#### Trend Analysis: 
Ploting time series of counts of tweets per day to see if there's any visible trend or seasonality.

In [ ]:
tweets_df.resample('D').size().plot(title='Daily Tweets')
plt.xlabel('Date')
plt.ylabel('Number of tweets')
plt.show()

#### Seasonality Check: 
Use seasonal decomposition to observe inherent seasonality in the data.

In [ ]:
!pip install statsmodels

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(tweets_df.resample('D').size(), model='additive')
result.plot()
plt.show()

### Correlation and Causation Analysis

#### Stationarity Check: 
Ensure the time series data is stationary, as this is a requirement for models like ARIMA.

In [ ]:
from statsmodels.tsa.stattools import adfuller
test_result = adfuller(daily_sentiment.dropna())
print('ADF Statistic: %f' % test_result[0])
print('p-value: %f' % test_result[1])

### Data Splitting: 
Decide on the training and testing periods for the models. Typically, the latter part of the data is held out for testing.

### Apache Spark

Setup the Notebook Environment.

Import the required libraries and set up the Spark session to interface with Cassandra and handle data processing needs:

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import os

# Set the SPARK_LOCAL_IP environment variable if necessary
# os.environ['SPARK_LOCAL_IP'] = '10.0.2.15' 

# Initialize a Spark session and Configure Spark to interact with Cassandra
spark = SparkSession.builder \
    .appName("ProjectTweets.csv") \
    .config("spark.master", "local") \
    .config("spark.ui.port", "4050") \
    .config("spark.cassandra.connection.host", "localhost") \
    .config("spark.cassandra.connection.port", "9042") \
    .getOrCreate()

#### Data Loading and Preliminary Analysis

First, load the CSV file "ProjectTweets.csv" without assuming it has headers. With specify header=False to tell Spark that the first row should not be treated as headers, and then it will explicitly provide the column names.

In [ ]:
tweets_df = spark.read.csv("file:///home/hduser/Desktop/INTEGRATEDCA2V2/ProjectTweets.csv", 
                                   header=False, inferSchema=True)
tweets_df.show(2, truncate=False)

### Data Cleaning and Preprocessing

Because this dataset lacks column titles, column names must be assigned manually when the data is loaded into Spark. This guarantees that the data processing and analysis processes are simple to comprehend and manage. 

Let's implement it properly,using the `toDF()` method as it given the data structure.

In [ ]:
# Define column names
column_names = ["id", "user_id", "date", "query_status", "user_handle", "tweet_text"]

# Rename the columns
tweets_df = tweets_df.toDF(*column_names)
tweets_df.show(5, truncate=False) # Using truncate=False allows to see the full content.

In [ ]:
# Print current DataFrame columns to understand the naming
print(tweets_df.columns)

In [ ]:
# Displaying dtypes of columns
tweets_df.dtypes

In [ ]:
type(tweets_df)

#### Verify the DataFrame
After renaming the columns, it’s a good idea to check the first few rows of the DataFrame to ensure that everything is loaded correctly.

In [ ]:
# Print DataFrame schema
tweets_df.printSchema()

tweets_df.show(5, truncate=True)

Based on the first details, the dataset appears to be a collection of tweets, commonly can be use in sentiment analysis or other natural language processing tasks. 

The dataset columns like 'id' and 'user_id' employ Integer data types, which are great for unique IDs and indexing. Meanwhile, columns labelled 'date', 'query_status', 'user_handle', and 'tweet_text' are of the String type, which can include a wide range of textual and category information.

#### Saving the DataFrame
Consider saving this DataFrame in a more effective format, such as Parquet, in case it needs to be reused with the correct column names in subsequent sessions or analyses.

In [ ]:
# Save the DataFrame for future use
tweets_df.write.parquet("/path/to/save/tweets_df.parquet", mode="overwrite")

# Load it whenever needed
tweets_df = spark.read.parquet("/path/to/save/tweets_df.parquet")

Saving a DataFrame in a format such as Parquet is extremely useful, particularly when dealing with huge datasets or when the same data will be reused across numerous sessions or projects. Here are some of the primary reasons why saving the DataFrame in a format like Parquet is recommended: Storage and I/O operations are more efficient, with built-in schema preservation and the ability to handle complex data types.

In [ ]:
# Count the number of records
print(f"Class: {type(tweets_df)}")
print(f"Total records: {tweets_df.count()}")

The dataset contains approximately 1.6 million entries. 
Scalability of the dataset: The dataset's 1.6 million items make it suitable for training increasingly sophisticated data-driven models, particularly deep learning models used in natural language processing and other machine learning applications.

#### Checking for Missing Values

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Get number of non-nulls per column by subtracting count of nulls from total entries
total_entries = tweets_df.count()
for column in tweets_df.columns:
    non_nulls = total_entries - tweets_df.filter(tweets_df[column].isNull()).count()
    print(f"Column '{column}' has {non_nulls} non-null entries")

In [ ]:
# Initialize a flag to track whether missing values are found
missing_values_found = False

for column in tweets_df.columns:
    missing_count = tweets_df.filter(col(column).isNull() | (col(column) == '')).count()
    if missing_count > 0:
        print(f"Column {column} has {missing_count} missing values")
        missing_values_found = True

# Check the flag after checking all columns, and print a message if no missing values were found
if not missing_values_found:
    print("No missing values found in any column.")

After checking each column in the dataset for null entries or empty strings, no such missing values were found across all columns, indicating that the dataset is complete and ready for further analysis without the requirement for data cleaning to fill or remove missing entries.

### Data Exploration (EDA)

In [ ]:
import pyspark
print(pyspark.__version__)

In [ ]:
import py4j
print(py4j.__version__)

In [ ]:
!pip install pyarrow

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
pandas_df = tweets_df.toPandas()

In [ ]:
# Display the first few rows of the DataFrame
print(pandas_df.head())

In [ ]:
# Summary statistics for numerical columns
print(pandas_df.describe())

# Check for missing values
print(pandas_df.isnull().sum())

In [ ]:
# Basic statistics
tweets_df.describe().show()

In [ ]:
# Displaying the values of a particular column
tweets_df.select('date').show(n=10, truncate=False)

In [ ]:
# Selecting multiple columns
tweets_df.select(['user_id','user_handle']).show(n=10, truncate=False)

### HandySpark

HandySpark is very helpful when moving from Pandas to PySpark since it creates a bridge by giving Spark DataFrames an interface similar to Pandas. Improved statistical analysis tools that are not easily accessible in PySpark, improved visualisation possibilities straight from DataFrames, and simpler, more intuitive data manipulation are all made possible by this connection. With features like stratified sampling, HandySpark also makes managing unbalanced datasets easier and provides improved methods for handling missing data. HandySpark retains the scalable architecture of Spark, even though it adds some overhead when compared to native PySpark. This makes it a viable option for individuals who want to combine the simplicity of Pandas with the capability of distributed computing.

In [ ]:
#pip install handyspark

#### Distributions
Examine the distribution of key variables using histograms or box plots to understand their central tendencies and spread.

#### Histograms: Useful for visualizing the distribution of numerical data.

In [ ]:
tweets_df.cache()

Caching can prevent recomputation of the DataFrame from the source data, improving performance for repeated queries and transformations.

In [ ]:
from handyspark import *
from pyspark.sql import SparkSession

# Create or get the existing Spark session
spark = SparkSession.builder.appName("User ID Histogram with HandySpark").getOrCreate()

# Convert your PySpark DataFrame to a HandySpark DataFrame
hdf = tweets_df.toHandy()

# Using HandySpark to plot a histogram
hdf.cols['user_id'].hist(bins=10, figsize=(8, 6), color='blue')

In [ ]:
numeric_data = tweets_df.select("user_id").rdd.flatMap(lambda x: x).histogram(10)
pd.DataFrame(list(zip(*numeric_data)), columns=["bin", "frequency"]).plot(kind="bar")

#### Box Plots: Good for detecting outliers and understanding the distribution's quartiles.

In [ ]:
tweets_df.toPandas().boxplot(column=['user_id'])

#### Pair Plots
Use pair plots to visualise the relationships and distributions of each pair of variables in the data.

In [ ]:
sampled_pd = tweets_df.select(numeric_features).sample(False, 0.1).toPandas()
sns.pairplot(sampled_pd)

#### Correlation Matrix
Investigate the correlations between numerical variables to better understand the links between the fields in the dataset.

In [ ]:
numeric_features = [t[0] for t in tweets_df.dtypes if t[1] == 'int' or t[1] == 'double']
sampled_data = tweets_df.select(numeric_features).sample(False, 0.1).toPandas()
correlations = sampled_data.corr()
sns.heatmap(correlations, annot=True)

### Categorical Data Analysis:

If the data contains categorical information, use bar or pie charts to visualise the distribution of categories.

In [ ]:
tweets_df.groupBy("some_categorical_column").count().toPandas().plot(kind='bar', x='some_categorical_column', y='count')


#### Pie Charts:

In [ ]:
pie_data = tweets_df.groupBy("another_categorical_column").count().toPandas()
pie_data.plot(kind='pie', y='count', labels=pie_data['another_categorical_column'], autopct='%1.1f%%')

#### Bar Charts:

In [ ]:
tweets_df.groupBy("some_categorical_column").count().toPandas().plot(kind='bar', x='some_categorical_column', y='count')


#### Time Series Analysis
For data with a temporal component, plot time series to see trends, cycles, or irregular patterns.

In [ ]:
time_series_data = tweets_df.groupBy("date_column").agg({"numerical_column": "mean"})
time_series_data.toPandas().plot(x='date_column', y='mean_numerical_column')

#### Text Data Analysis
For datasets containing text data, such as tweets, conduct text-specific analyses:

#### Word Frequency Count:

In [ ]:
from pyspark.sql.functions import explode, split
tweets_df.withColumn("word", explode(split(tweets_df["text"], "\s+"))).groupBy("word").count().orderBy("count", ascending=False).show()

#### Word Cloud:

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text_data = tweets_df.select("text").rdd.flatMap(lambda x: x).collect()
text_string = " ".join(text_data)
wordcloud = WordCloud().generate(text_string)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Data Preparation

Prepare the data for analysis, which may include parsing dates or extracting specific features:

In [ ]:
from pyspark.sql.functions import to_date

# Convert 'date' to a proper date format if needed
clean_tweets_df = clean_tweets_df.withColumn("date", to_date("date"))

### Sentiment Analysis 

Choose Sentiment Analysis Technique:
Depending on the libraries available, setup for sentiment analysis might involve using an NLP library or building a custom model:

In [ ]:
# Example using TextBlob for sentiment analysis (simplistic and illustrative)
from textblob import TextBlob

def sentiment_analysis(text):
    return TextBlob(text).sentiment.polarity

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

sentiment_udf = udf(sentiment_analysis, FloatType())

# Apply sentiment analysis
sentiment_df = clean_tweets_df.withColumn("sentiment", sentiment_udf(clean_tweets_df["text"]))
sentiment_df.show(5)

### Time Series Analysis and Forecasting

#### Aggregate Data for Time Series:
Prepare the data for time series analysis, typically needing aggregation by date

In [ ]:
from pyspark.sql.functions import avg

daily_sentiment = sentiment_df.groupBy("date").agg(avg("sentiment").alias("avg_sentiment"))
daily_sentiment.show()

#### Forecasting:
Implement time series forecasting models such as ARIMA or LSTM

#### Visualization and Reporting
Dynamic Dashboard Creation:
Use an appropriate tool like Plotly or PowerBI to create an interactive dashboard. Prepare the data in notebook and export it for visualization.

Documentation:
Document the findings, methods, and justifications in the report according to the project guidelines.

References:

Apache Parquet Documentation
The official documentation for the Parquet file format offers insights into its design, features, and benefits for using it in data storage and processing tasks.https://spark.apache.org/docs/latest/

Databricks Resources
Databricks, a company founded by the creators of Apache Spark, provides extensive resources, blogs, and tutorials on Spark and Parquet, including best practices for performance optimization.
Link: Databricks - Apache Spark Resources

This book by Holden Karau, Andy Konwinski, Patrick Wendell, and Matei Zaharia (O'Reilly Media) is a great resource to learn about Spark from the ground up, covering basic to advanced topics.
ISBN: 978-1449358624